### General code for converting fastq to fasta and running BLAST
* opens fastq file and converts it to fasta by writing the first 2 of 4 lines
* BLASTs it using NCBIWWW
* writes result into `name_res.xml`

In [2]:
# import packages for BLAST API
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import csv
import re

filename = "ERR4775534"
filepath = "data/ibs/"
resultspath = "results/ibs/"

In [3]:
# fastq to fasta
file = open(filepath + filename + ".fastq", "r")
file_fasta = open(filepath + filename + ".fasta", "w")
lines = file.readlines()

counter = 1

for line in lines:
    if counter == 1:
        file_fasta.write(">")
        file_fasta.write(line)
    if counter == 2:
        file_fasta.write(line)
    counter = (counter + 1) % 4

file.close()
file_fasta.close()

In [5]:
sequence_data = open(filepath + filename + ".fasta", "r").read()
result_handle = NCBIWWW.qblast("blastn", "nt", sequence_data, format_type='HTML') 
result_handle

ValueError: No RID and no RTOE found in the 'please wait' page, there was probably an error in your request but we could not extract a helpful error message.

In [4]:
results = open(resultspath + filename + "_res.html", "w")
blast_results = result_handle.read()
results.write(blast_results)

290709

In [27]:
# parse BLAST result (html version)

results_html = open(resultspath + filename + "_res.html", "r")

file = open(resultspath + filename + "_res_parsed.csv", "w", newline='')
file.truncate(0)
writer = csv.writer(file)
lines = results_html.readlines()

writer.writerow(["taxid", "count"])

for line in lines:
    split = line.split()
    # print(split)
    if "taxid" in split:
        for i in range(len(split)):
            if split[i] == "taxid":
                if split[i + 1] != "or":
                    res = re.findall('\d+', split[i+1])
                    writer.writerow([res[0], 1])

results_html.close()
file.close()
file = open(resultspath + filename + "_res_parsed.csv", "r", newline='')
reader = csv.reader(file)
header = []
header = next(reader)
rows = []
for row in reader:
    rows.append(row)

output_dict = {}
for row in rows:
    if row[0] in output_dict:
        output_dict[row[0]] += 1
    else:
        output_dict[row[0]] = 1

file.close()
print(output_dict)

file = open(resultspath + filename + "_info.csv", "w", newline='')
writer = csv.writer(file)
for key in output_dict:
    writer.writerow([filename, key, output_dict[key]])
file.close()


{'305': 3, '1937813': 1, '235299': 1, '562': 8, '1501662': 1, '2026743': 1, '4615': 1, '1747': 27, '267747': 1, '1122995': 1, '1734925': 2, '1869227': 2, '1280': 1}
